# Experiment Configuration and Execution

In this tutorial we show how to configure and execute an experiment in OpenSBT to test an ADS Agent. We show the configuration of the experiment for the simulation of an agent in the DummySimulator (simplistic SUT & Simulation, no GPU required).



# Description Use Case

The dummy simulations use case contains an AEB agent and another pedestrian. Both actors move linear, while pedestrian has a fixed trajectory and fixed velocity, the ego agent has a predefined path but can modife his velocity. I.e. he brakes when he detects other actors within a predefined range.
We want to test the AEB agent in violating the safety distance of 1m. For this we want to vary the initial velocities of both actors as well the orientation of the pedestrian crossing the agents lane.

In particlar, the pedestrian has a speed in the range of [0.2, 3] m/s, the ego [1, 10] m/s, while orientation range is [0,180] degrees.

### Defining An Experiment

To define an experiment we need to instantiate `ADASProblem`.

ADASProblem holds:

- the **scenario** - which can be an OpenSCENARIO file, or file of another format supported by simulator
- the **search variables** - whose values are altered to produce (different) scenario througout the search
- the **search space** represented by the lower and upper bounds as arrays
- the **fitness function** 
- the **criticality function** - represents the safety requirements/testing oracle
- the **simulate function** - the simulator is triggered via this function
- the **simulator-related variables**, i.e., simulation time, sampling time, toggle for visualization

Note, that for now the SUT is defined via the simulator functions, as we assume that the SUT is embedded into the simulator. We are working on an update where the SUT can be define uncoupled from the simulator.

We 


```python

from problem import ADASProblem

problem = ADASProblem(
                        problem_name="DummySimulatorProblem,
                        scenario_path="/scenarios/DummyScenario.xosc",
                        xl=[0.5, 1, 0],
                        xu=[3, 80, 60],
                        simulation_variables=[
                            "PedestrianSpeed",
                            "FinalHostSpeed",
                            "PedestrianEgoDistanceStartWalk"],
                        fitness_function=MyFitnessFunction(),  
                        critical_function=MyCriticalityFunction(),
                        simulate_function=MySimulator.simulate,
                        simulation_time=10,
                        sampling_time=100,
                        do_visualize = False
                        )
                        
```



### Executing An Experiment From Code

We have two options to execute an experiment: via console or via the code directly.

To start search without console, we instantiate the optimizer class of our corresponding algorithm and can directly run the search. To write the results, we can pass to the corresponding writing method the optimization paramaters. The results will be written in the default folder name `results`. A different folder name can be also passed.

```python
# Instantiate search algorithm
optimizer = NsgaIIOptimizer(
                            problem=problem,
                            config=DefaultSearchConfiguration()
                            )
# Run search
res = optimizer.run()

# Write results

res.write_results(params = optimizer.parameters)
```



### Starting Search Using Console

For this option as next step we need to create an `Experiment` instance, where we specify the algorithm to be used for the search and its configuration.
The experiment requires following inputs:

- the name
- the created problem, 
- the search algorithm and 
- the search configuration

Consider, the algorithm is an enumeration value which need to be registed beforehand via the algorithm.py script.

```python
experiment = Experiment(name="1",
                        problem=problem,
                        algorithm=AlgorithmType.NSGAII,
                        search_configuration=DefaultSearchConfiguration())
```

Then we register the experiment so that we can use it later via console.

```python
experiments_store.register(experiment)
```

To start search via console we run the experiment with the name "1":

```bash
python run.py -e 1
```

We can change the experiment parameter as e.g., lower and upper bounds of the search parameters and the search time by using flags:


```bash
python run.py -e 1 -min 0 0 -max 10 2 -m  "FinalHostSpeed" "PedestrianSpeed" -t "01:00:00"
```

The complete list of flags is available (here)[].
